<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://raw.githubusercontent.com/logpai/logparser/master/logs/HDFS/HDFS_2k.log'


--2023-08-31 13:11:04--  https://raw.githubusercontent.com/logpai/logparser/master/logs/HDFS/HDFS_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287848 (281K) [text/plain]
Saving to: ‘HDFS_2k.log’

HDFS_2k.log         100%[===================>] 281.10K  --.-KB/s    in 0.04s   

2023-08-31 13:11:04 (7.34 MB/s) - ‘HDFS_2k.log’ saved [287848/287848]



In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2ec907ae9d059624b9e8cbd0d50a9e0fea23b6f7f2d7b754098e0561704b68d1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import re
import string
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.decomposition import PCA

In [31]:
def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message

    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
text = "081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating"

pattern = r"blk_\d+"  # Regular expression pattern to match "blk_" followed by digits
match = re.search(pattern, text)
print(match)
print(match.group())

<re.Match object; span=(81, 102), match='blk_38865049064139660'>
blk_38865049064139660


In [50]:
def find_blockid(text):
  match = re.search(r"blk_[-\w]+", text)
  return match.group()

In [17]:
find_blockid(text)

'blk_38865049064139660'

In [52]:
with open('/content/HDFS_2k.log', "r") as file:

    log2index = {}    # give for each unique cleaned log an index
    index2embed = {}  # give for each index(belongs to a unique cleaned log) sentence embedding of that unique cleaned log
    blockId_logs = {} # gather logs(index) which are blong to a specific block id
    index = 0

    for line in file:

      blockId = find_blockid(line)
      if not blockId in blockId_logs:
            blockId_logs[blockId] = []

      cleaned_line = clean(line)

      if log2index.get(cleaned_line) is None:
          log2index[cleaned_line] = index
          index2embed[index] = model.encode(cleaned_line)
          blockId_logs[blockId].append(index)
          index += 1
      else :
          blockId_logs[blockId].append(index)


In [53]:
len(blockId_logs)

1994

In [55]:
len(log2index)

14

In [56]:
len(index2embed)

14

In [68]:
# Step 1: Prepare Data
embeddings = np.array(list(index2embed.values()))


# Step 3: Apply PCA
num_components = 0.9  # Choose the number or percentage of variance of components you want
pca = PCA(n_components=num_components)
pca.fit(embeddings)

# Step 4: Transform Data
reduced_embeddings = pca.transform(embeddings)

print(reduced_embeddings.shape)

(14, 8)


In [69]:
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))


[0.31995746 0.1347681  0.1056298  0.09456902 0.08474008 0.07934943
 0.05515406 0.03950324]
0.9136712


In [70]:
reduced_index2embed = {}
for i,key in enumerate(index2embed.keys()):
  reduced_index2embed[key]=reduced_embeddings[i]

In [71]:
reduced_index2embed

{0: array([-0.17603853, -0.34237605,  0.11730315, -0.10971322,  0.16866136,
         0.15810293, -0.18084237, -0.14752673], dtype=float32),
 1: array([ 0.3376382 , -0.0314443 , -0.31014043,  0.10282857, -0.08823918,
         0.19950257,  0.23421362,  0.02333169], dtype=float32),
 2: array([-0.21855204, -0.350302  , -0.19773616,  0.07163797,  0.15267113,
         0.18293613, -0.06290735,  0.10659716], dtype=float32),
 3: array([-0.37509143,  0.14105031, -0.11613685,  0.03709174, -0.04770277,
        -0.22551155, -0.04183104, -0.08920407], dtype=float32),
 4: array([ 0.40750003,  0.02095165, -0.14684449, -0.0864957 , -0.1961211 ,
         0.02211149, -0.05259257, -0.15011781], dtype=float32),
 5: array([-0.02226402, -0.03324205,  0.36332995,  0.4914382 , -0.16707622,
         0.07700343,  0.01990281, -0.04711287], dtype=float32),
 6: array([ 0.25061953,  0.06138339,  0.11468643,  0.10801424,  0.367491  ,
        -0.224677  ,  0.11108578,  0.06684642], dtype=float32),
 7: array([-0.362039